## Какие преимущества VIP статусов Binance могут быть полезны компании алгоритмического трейдинга

### Задача - выведение оптимальной стратегии подъема статуса с общего пользователя до 9 VIP-статуса на Binance для компании алгоритмического трейдинга на примере Spot-торговли

План действий:

1. Выгрузка данных по url
2. Преобразования данных
3. Поиск оптимальной комбинации повышения статусов для Maker\
    3.1. Пул комбинаций статусов\
    3.2. Максимальный объем\
    3.3. Механика расчета
4. Расчет оптимальной стратегии подъема статуса для Taker

In [15]:
#!pip install selenium
#!pip install beautifulsoup4 

In [1]:
#import requests 
import pandas as pd
import numpy as np
from selenium import webdriver # используем selenium из-за динамической подгрузки данных на сайте
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from scipy.special import comb
from itertools import combinations

## 1. Выгрузка данных по url

Binance предлагает 9 видов использования VIP-статса:

1. Trader program - Spot Trading
2. Trader program - USD
3. Trader program - COIN-M Futures (По существу она такая же как USD)
4. Trader program - ОТС
5. Holder program - Path A
6. Holder program - Path B (он зависит от Trader VIP статуса)
7. Borrower program
8. Investor program - Simple earn
9. Investor program - Dual investment program


Какие из этих VIP-программ интересны для компании алгоритмического трейдинга?
Первые 4, где описаны "Trader program". По сути основными преимуществами для алгоритмического трейдинга будут низкие комиссии, что и предоставляют VIP-статусы. Также комиссию можно снизить торгуя BNB. Любопытным моментом здесь является то, как можно повысить статус с 0 до 9 с наименьшими потерями - это мы и постараемся выяснить.

Начнем с выгрузки данных с сайта. Исходя из кода страницы, нужная нам информация находится в таблицах, которые переключаются по нажатию (т.е. динамическая подгрузка) поэтому используем для выгрузки таблиц библиотеку selenium. Выгрузим первую интересующую нас таблицу со Spot ставками

In [22]:
# Запускаем браузер, заходим на страницу
url = 'https://www.binance.com/en/fee/trading'
chrome_path = "C:\Program Files\Google\Chrome\chromedriver.exe"
driver = webdriver.Chrome(executable_path=chrome_path)
driver.get(url)

# Ждем, пока элемент станет видимым - иначе данные не подгружаются
driver.implicitly_wait(5)

# Получаем HTML-код страницы
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Находим 1ую таблицу
table = soup.find('table')

# Находим все строки в теле таблицы
rows = table.find('tbody').find_all('tr') # находим столбцы по индикатору tr

# Пропускаем первую строку с заголовками иначе как будто нет данных
data_rows = [] # для финального сбора данных
for row in rows[1:]:
    columns = row.find_all('td') # находим столбцы по индикатору td
    data_row = [column.text.strip() for column in columns] # данные из столбцов через запятую в списке
    data_rows.append(data_row)

# Создаем DataFrame
spot_df = pd.DataFrame(data_rows) # получаем полный список с данными

# Собираем заголовки таблицы из маркеров tr и th, п отом очищаем от лишней информации, добавляем в DataFrame
header = soup.find('thead').find('tr').find_all('th')
header_titles = [h.text.strip() for h in header]
spot_df.columns = header_titles

# Выводим DataFrame
display(spot_df)

# Закрываем браузер
driver.quit()

<>:3: SyntaxWarning: invalid escape sequence '\P'
<>:3: SyntaxWarning: invalid escape sequence '\P'
C:\Users\licen\AppData\Local\Temp\ipykernel_15252\4022185646.py:3: SyntaxWarning: invalid escape sequence '\P'
  chrome_path = "C:\Program Files\Google\Chrome\chromedriver.exe"
C:\Users\licen\AppData\Local\Temp\ipykernel_15252\4022185646.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_path)


,Level,30-Day Trade Volume (USD*)Trade volume will be converted into USD equivalent values based on the exchange rate under Multi-Asset mode.Learn more,and/or,BNB Balance,"Maker / TakerA “Taker” is a trader who places an order at the market price, whereas a “Maker” is a trader who places an order at a limit price.Learn More.",Maker / TakerBNB 25% off
0,Regular User,"< 1,000,000 USD",or,≥ 0 BNB,0.1000% / 0.1000%,0.0750% / 0.0750%
1,VIP 1,"≥ 1,000,000 USD",and,≥ 25 BNB,0.0900% / 0.1000%,0.0675% / 0.0750%
2,VIP 2,"≥ 5,000,000 USD",and,≥ 100 BNB,0.0800% / 0.1000%,0.0600% / 0.0750%
3,VIP 3,"≥ 20,000,000 USD",and,≥ 250 BNB,0.0420% / 0.0600%,0.0315% / 0.0450%
4,VIP 4,"≥ 100,000,000 USD",and,≥ 500 BNB,0.0420% / 0.0540%,0.0315% / 0.0405%
5,VIP 5,"≥ 150,000,000 USD",and,"≥ 1,000 BNB",0.0360% / 0.0480%,0.0270% / 0.0360%
6,VIP 6,"≥ 400,000,000 USD",and,"≥ 1,750 BNB",0.0300% / 0.0420%,0.0225% / 0.0315%
7,VIP 7,"≥ 800,000,000 USD",and,"≥ 3,000 BNB",0.0240% / 0.0360%,0.0180% / 0.0270%
8,VIP 8,"≥ 2,000,000,000 USD",and,"≥ 4,500 BNB",0.0180% / 0.0300%,0.0135% / 0.0225%
9,VIP 9,"≥ 4,000,000,000 USD",and,"≥ 5,500 BNB",0.0120% / 0.0240%,0.0090% / 0.0180%


## 2. Преобразования данных

Приведем таблицу к удобному для работы формату

In [23]:
spot_df = spot_df.drop('and/or', axis=1) # убираем неинформативные столбцы
spot_df.columns = ['Level',	'Trade Volume', 'BNB Balance',	'Commission', 'Commission_25%_off']
spot_df['Level'] = range(10) # оставляем в статусах и объеме только цифры
spot_df['Trade Volume'] = spot_df['Trade Volume'].replace(to_replace=r'\D+', value='', regex=True)
spot_df.iloc[0, 1] = 0 #поправляем необходимый объем для новичков
spot_df[['Com_Maker', 'Com_Taker']] = spot_df['Commission'].str.replace('%', '').str.split('/', expand=True).astype(float)
spot_df[['Com_Maker_25%_off', 'Com_Taker_25%_off']] = spot_df['Commission_25%_off'].str.replace('%', '').str.split('/', expand=True).astype(float)
spot_df = spot_df.drop(['Commission', 'Commission_25%_off'], axis=1)
display(spot_df)

,Level,Trade Volume,BNB Balance,Com_Maker,Com_Taker,Com_Maker_25%_off,Com_Taker_25%_off
0,0,0,≥ 0 BNB,0.100,0.100,0.0750,0.0750
1,1,1000000,≥ 25 BNB,0.090,0.100,0.0675,0.0750
2,2,5000000,≥ 100 BNB,0.080,0.100,0.0600,0.0750
3,3,20000000,≥ 250 BNB,0.042,0.060,0.0315,0.0450
4,4,100000000,≥ 500 BNB,0.042,0.054,0.0315,0.0405
5,5,150000000,"≥ 1,000 BNB",0.036,0.048,0.0270,0.0360
6,6,400000000,"≥ 1,750 BNB",0.030,0.042,0.0225,0.0315
7,7,800000000,"≥ 3,000 BNB",0.024,0.036,0.0180,0.0270
8,8,2000000000,"≥ 4,500 BNB",0.018,0.030,0.0135,0.0225
9,9,4000000000,"≥ 5,500 BNB",0.012,0.024,0.0090,0.0180


## 3. Поиск оптимальной комбинации повышения статусов для Maker

Как мы видим, чем выше статус, тем ниже ставка, но чтобы достичь этого статуса нужно соверщить операций на сумму, требуемого объема (Trade Volume) по менее выгодной ставке. Представим, что мы только зашли на биржу, т.е. мы находимся на 0 статусе, но мы собираемся выйти на огромные объемы (выше, чем требования для 9 VIP-статуса), как с наименьшими потерями нам этого достичь с учетом того, что статус пересматривается раз в месяц по итогам торгов этого месяца? 

1 вариант - Мы можем перескочить сразу с 0 на 9, реализовав 4000000000$ по ставке 0.1%. \
2 вариант - Можем идти постепенно т.е. 1000000 по ставке 0.1%, потом 5000000по ставке 0.09% и т.д., тогда последние 4000000000 будут уже выполняться по ставке 0.018%, а не 0.1% \
3.... варианты - Но есть еще масса промежуточных вариантов, где происходят перескоки с одного статуса на другой, но более плавно, чем в 1 варианте. (например 0,4,7,9 или 0,1,2,5,7,8,9)

**Наша задача понять, при каком сценарии мы потеряем совокупно меньше всего средств на комиссии.** Разобьем эту задачу на подзадачи:
1. Найти все возможные комбинации, которые начинаются на 0, заканчиваются на 9, идут строго в порядке возрастания, но возможно любое количество пропусков.
2. Чтобы при любом количестве итераций (9 если идти последовательно, 2 - если перескочить за раз с 0 на 9, или любом промежуточном) у нас коррекотно считалась средняя ставка, нужно отталкиваться от фиксированного объема средств, который мы "тратим" на любой сценарий. По сути нам надо взять сумму, необходимую для реализации максимально длинного сценария, т.е. того, где мы последовательно переходим каждый месяц на новый уровень.
3. Для каждого сценария из пункта 1. посчитать среднюю ставку при реализации полного объема (из пункта 2)

### 3.1. Пул комбинаций статусов

Решим эту задачу, найдя все подпоследовательности элементов множества {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}, начинающихся с 0 и заканчивающихся на 9. Используем библиотеку itertools для генерации всех комбинаций различных размеров от 1 до 9.

In [24]:
def generate_combinations():
    numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    all_combinations = []
    result_combinations = []

    for r in range(2, len(numbers) + 1): # задаем количество комбинаций
    # Функция combinations возвращает все комбинации указанного размера (от 2 до 9) из заданного списка
        all_combinations.extend(combinations(numbers, r)) 
    # Теперь отсортируем только те, что соответсвуют требованиям    
    for combination in all_combinations:
        if combination[0] == 0 and combination[-1] == 9:
            result_combinations.append(combination)
    return result_combinations

our_combinations = generate_combinations()
our_combinations


[(0, 9),
 (0, 1, 9),
 (0, 2, 9),
 (0, 3, 9),
 (0, 4, 9),
 (0, 5, 9),
 (0, 6, 9),
 (0, 7, 9),
 (0, 8, 9),
 (0, 1, 2, 9),
 (0, 1, 3, 9),
 (0, 1, 4, 9),
 (0, 1, 5, 9),
 (0, 1, 6, 9),
 (0, 1, 7, 9),
 (0, 1, 8, 9),
 (0, 2, 3, 9),
 (0, 2, 4, 9),
 (0, 2, 5, 9),
 (0, 2, 6, 9),
 (0, 2, 7, 9),
 (0, 2, 8, 9),
 (0, 3, 4, 9),
 (0, 3, 5, 9),
 (0, 3, 6, 9),
 (0, 3, 7, 9),
 (0, 3, 8, 9),
 (0, 4, 5, 9),
 (0, 4, 6, 9),
 (0, 4, 7, 9),
 (0, 4, 8, 9),
 (0, 5, 6, 9),
 (0, 5, 7, 9),
 (0, 5, 8, 9),
 (0, 6, 7, 9),
 (0, 6, 8, 9),
 (0, 7, 8, 9),
 (0, 1, 2, 3, 9),
 (0, 1, 2, 4, 9),
 (0, 1, 2, 5, 9),
 (0, 1, 2, 6, 9),
 (0, 1, 2, 7, 9),
 (0, 1, 2, 8, 9),
 (0, 1, 3, 4, 9),
 (0, 1, 3, 5, 9),
 (0, 1, 3, 6, 9),
 (0, 1, 3, 7, 9),
 (0, 1, 3, 8, 9),
 (0, 1, 4, 5, 9),
 (0, 1, 4, 6, 9),
 (0, 1, 4, 7, 9),
 (0, 1, 4, 8, 9),
 (0, 1, 5, 6, 9),
 (0, 1, 5, 7, 9),
 (0, 1, 5, 8, 9),
 (0, 1, 6, 7, 9),
 (0, 1, 6, 8, 9),
 (0, 1, 7, 8, 9),
 (0, 2, 3, 4, 9),
 (0, 2, 3, 5, 9),
 (0, 2, 3, 6, 9),
 (0, 2, 3, 7, 9),
 (0, 2, 3, 8, 9),
 (0, 2,

Проверим с помощью комбинаторики. В данном случае, задача сводится к нахождению всех подпоследовательностей элементов множества {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}, начинающихся с 0 и заканчивающихся на 9. Но при этом 0 и 9 фиксированы по месту, поэтому их проще из расчета исключить и добавить в конце.

Таким образом, общее количество подпоследовательностей будет равно:

C(1,8)+C(2,8)+C(3,8)+C(4,8)+C(5,8)+C(6,8)+C(7,8)+C(8,8)\
где C(n,k) - число сочетаний, и плюс одно краевое значение (0,9)

In [25]:
# т.к. 0 и 9 у нас фиксированы, то нужно считать комбинации от 1 до 8 и прибавыить одну крайнюю комбинацию (0,9)
combination_number = sum(comb(8, r) for r in range(1, 9)) 
print((combination_number+1)==len(our_combinations))
len(our_combinations)

True


256

Ура, мы нашли все возможные комбинации роста статусов

### 3.2. Максимальный объем
Рассуждаем так: когда каждый сценарий достигает 9ого статуса, после этого комиссия будет минимальной у всех, поэтому ее в сравнительный расчет не имеет смысла брать. Однако тормозить всех на достижении 9ого статуса будет не честно, поскольку при более быстром достижении, мы платим изначально большую сумму по невыгодной ставке, но дальше она становится максимально выгодной. Следовательно, для честного сравнения надо использовать какой-то фиксированный объем, на котором мы посчитаем среднюю ставку для всех сценариев. Максимальный объем у нас тратится в случае, когда статус поднимаем каждый месяц постепенно, следовательно за максимальный объем возьмем сумму требуемого объема всех статусов

In [26]:
spot_df['Trade Volume'] = spot_df['Trade Volume'].astype(dtype=np.int64)
max_trade_volume = spot_df['Trade Volume'].sum()
max_trade_volume

7476000000

Этот объем будем использовать для расчета ставки в каждой транзакции

### 3.3. Механика расчета
Посмотрим на несколько примеров, как мы будем считать суммарную комиссию, которую мы отдадим бирже. Сразу отметим, что будем рассматривать ситуации, когда мы торгуем предельно на тот объем, чтобы перескочить на более высокий статус, другие варианты априори менее эффективные.

1. В самом длинном случае мы берем объем необходимый для 1 статуса и умножаем на ставку 0-ого статуса, потом прибавляем объем для 2-ого статуса умноженный на ставку 1-ого и т.д. Т.е. ставка следующей итерации зависит от текущего объема , что логично. Схематично это можно изобразить так:

V1*t0 + V2*t1 +....V9*t8

в данном случае мы реализуем весь объем max_trade_volume как раз до того как достигнем оптимальной ставки. условно можно записать последний аргумент как + (max_trade_volume-(V1+V2+...+V8+V9))*t9, но он равен 0

2. Другой вариант - самый короткий, когда мы переходим сразу на 9 уровень:

V9*t0 + (max_trade_volume-V9)*t9

в этом случае мы сразу перескакиваем на 9 статус и реализуем максимальный объем с самой низкой ставкой

3. Промежуточные варианты

V1*t0 + V3*t1 + V8*t3 + V9*t8 + (max_trade_volume-(V1+V3+V8+V9))*t9

В этом случае мы как бы перескакиваем через некоторые статусы, и за счет этого можем реализовать некоторое количество по оптимальной ставке

Теперь реализуем этот подсчет в коде

In [27]:
def find_optimal_combination(max_trade_volume, rates, volumes, our_combinations):
    best_combination = None
    min_total_fee = float('inf')

    for combination in our_combinations:
        #ставки и объемы фиксируем со сдвигом, т.к. чтобы перейти на новый статус нужен объем нового статуса, а ставка текущего
        selected_rates = [rates[j] for j in combination][:-1]
        selected_volumes = [volumes[j] for j in combination][1:]
        # Фиксируем ставку последнего статуса для оставшегося объема в конце
        rate_V_9 = [rates[j] for j in combination][-1]
        
        total_fee = 0
        remaining_volume = max_trade_volume

        for rate, volume in zip(selected_rates, selected_volumes):
            # Перемножаем объемы на ставки со сдвигом, который мы задали выше
            total_fee += volume * rate
            # Считаем, какой объем остался от максимального
            remaining_volume -= volume
        # Добавляем его по ставке 9-ого статуса
        total_fee += remaining_volume * rate_V_9
        #print('Current combination and fee: ', combination, total_fee)

        if total_fee < min_total_fee:
            min_total_fee = total_fee
            best_combination = combination

    return best_combination, min_total_fee

In [28]:
rates = spot_df['Com_Maker'].tolist()
volumes = spot_df['Trade Volume'].tolist()

best_combination, min_total_fee = find_optimal_combination(max_trade_volume, rates, volumes, our_combinations)

print(f"Best Combination: {best_combination}")
print(f"Min Total Fee: {min_total_fee}")
print(f"Min Total Fee in %: {min_total_fee/max_trade_volume}")

Best Combination: (0, 1, 3, 5, 7, 9)
Min Total Fee: 163060000.0
Min Total Fee in %: 0.021811128945960406


Супер, мы выяснили, что если используем только стратегию Maker, то оптимальным темпом поднятия статуса будет: 0, 1, 3, 5, 7, 9. И минимальный fee, уплаченный бирже составит 163060000$, что составляет 2.18% от общей вложенной суммы. После достижения 9 статуса, fee уже будет минимальным и средняя ставка будет снижаться. Безусловно, расчет комиссий упрощен и не учитывает другие факторы, такие как объем торговли, частота сделок и динамика рынка. Также мы не закладываем сюда ожидаемую доходность от актива, и тот факт, что чем дольше мы будем идти к 9 статусу, тем больше альтернативных вложений потеряем. Однако подобный подход позволил математически расчитать самый выгодный способ повышения уровня для Maker стратегии, расчитаем такой же для Taker стратегии

## 4. Расчет оптимальной стратегии подъема статуса для Taker

In [29]:
rates = spot_df['Com_Taker'].tolist()
volumes = spot_df['Trade Volume'].tolist()

best_combination, min_total_fee = find_optimal_combination(max_trade_volume, rates, volumes, our_combinations)

print(f"Best Combination: {best_combination}")
print(f"Min Total Fee: {min_total_fee}")
print(f"Min Total Fee in %: {min_total_fee/max_trade_volume}")

Best Combination: (0, 3, 5, 7, 9)
Min Total Fee: 253544000.0
Min Total Fee in %: 0.03391439272338149


Для Taker оптимальной стратегией будет перескочить 1 уровень (вероятно из-за того, что для Taker VIP-0 и VIP-1 имеют одинаковую ставку комиссии) и суммарная комиссия, уплаченная бирже составит 3,39%, что в целом ожидаемо выше, чем у Maker потому что Taker-операции снижают ликвидность. Аналогично можно использовать эту функцию для USD или фьючерсов.

Также в дальнейшем используя эту же функцию можно просчитывать и более сложные стратегии, например различные комбинации Maker-Taker, или для других рынков. Задача будет только в том, чтобы правильно расчитать ожидаемую доходность и скорректировать столбец rates для расчетов.